# HyperSpy入門 RDEデータ登録デモンストレーション
HyperSpyに関する講義の最後に、dm3ファイルをRDEにアップロードしてデータ登録を行うデモンストレーションをご覧いただきます。

# ARIM事業におけるRDEとPythonの関係

<img src="https://github.com/tendo-sms/python_intermediate_2023/raw/main/file_4/figure/rde.png" width="70%" alt="ARIM事業におけるRDEとPythonの関係">

ARIM事業の中で活用しているRDE(Research Data Express)は、材料研究のために研究現場で日々創出される材料データを効率的に収集するために、次の機能を備えたデータ収集・管理・提供システムのことです。

* IoT データ転送機能
* データ構造化機能
* データ登録機能

データ構造化機能は、Pythonで開発が進められています。

# 使用するデータ構造化プログラム
今回のデモンストレーションのため、講義でご紹介した「dm3ファイルの読み込み例」を改良した構造化プログラムを、RDE上にセットアップしてあります。

## 講義内容のおさらい
「dm3ファイルの読み込み例」の内容をおさらいしておきましょう。

## 事前準備

In [ ]:
# dm3ファイルのダウンロード
!curl -L -o Fei_HAADF-DE_location.dm3 https://github.com/tendo-sms/python_intermediate_2023/raw/main/file_2/Fei_HAADF-DE_location.dm3

# HyperSpyパッケージのインストール
!pip install HyperSpy

## プログラム
### メタデータの内容を表示する

In [ ]:
import hyperspy.api as hs

# dm3ファイルの読み込みとメタデータの表示
s = hs.load("Fei_HAADF-DE_location.dm3")
s.metadata

### グラフを描画する

In [ ]:
import hyperspy.api as hs

# dm3ファイルの読み込みとグラフの描画
s = hs.load("Fei_HAADF-DE_location.dm3")
s.plot()

## RDE上にセットアップしたプログラムについて
上記のプログラムをベースに、構造化プログラムをRDE上にセットアップしてあります。

このプログラムは、次の構造化を行います。

* metadataの内容をRDEのメタデータとして登録する
* グラフをPNG画像としてRDEに登録する

講義でご紹介した「dm3ファイルの読み込み例」はごく簡単なプログラムですが、RDE上で動かすために、様々な改修をしています。

データ登録のデモンストレーションがメインであるため、ここでは構造化のポイントとなる部分を抜粋してご紹介します。

## metadataの内容をRDEのメタデータとして登録する
HyperSpyのデータから、metadataに格納されたメタデータの値を取得するには、<font color="red">**get_itemメソッド**</font>を使用します。

get_itemメソッドの引数には、メタデータの項目名を文字列で指定します。

~~~
オブジェクト.get_item(項目名)
~~~

metadataの内容は階層構造になっているため、項目名は、要素をピリオドで連結することで階層を表現します。(ただし、要素名が数字Nの場合は「Number_N」と指定します。)

試しに、metadataに格納されたメタデータの値をいくつか取得してみます。取得した結果は、項目名と値をペアにした辞書に格納します。

In [ ]:
import hyperspy.api as hs

s = hs.load("Fei_HAADF-DE_location.dm3")

# メタデータ項目名のリスト
METTAKEY_LIST = [
    "Acquisition_instrument.TEM.Stage.tilt_alpha",
    "General.FileIO.Number_0.hyperspy_version"  # 要素"0"を「Number_0」とする点に注意
    ]

# メタデータ項目の値の取得　結果は辞書に格納する
data = {}
for metakey in METTAKEY_LIST:
    data[metakey] = s.metadata.get_item(metakey)

print(data)

# RDEに登録するときはJSONファイル(metadata.json)にしてmetaディレクトリに配置する必要がある



今回、この階層の要素をピリオドで連結した項目名を、そのままRDEに登録するメタデータの項目名とします。一覧を次に示します。

* Acquisition_instrument.TEM.Stage.tilt_alpha
* Acquisition_instrument.TEM.Stage.tilt_beta
* Acquisition_instrument.TEM.Stage.x
* Acquisition_instrument.TEM.Stage.y
* Acquisition_instrument.TEM.Stage.z
* Acquisition_instrument.TEM.acquisition_mode
* Acquisition_instrument.TEM.beam_current
* Acquisition_instrument.TEM.beam_energy
* Acquisition_instrument.TEM.camera_length
* Acquisition_instrument.TEM.dwell_time
* Acquisition_instrument.TEM.magnification
* General.FileIO.Number_0.hyperspy_version
* General.FileIO.Number_0.io_plugin
* General.FileIO.Number_0.operation
* General.FileIO.Number_0.timestamp
* General.date
* General.original_filename
* General.time
* General.title
* Signal.Noise_properties.Variance_linear_model.gain_factor
* Signal.Noise_properties.Variance_linear_model.gain_offset
* Signal.quantity
* Signal.signal_type

辞書が完成したら、最終的に<font color="red">**metadata.json**</font>という名称のJSONファイルを、metaという名前のディレクトリに格納します。

このmetadata.jsonと、あらかじめデータセットごとにメタデータ項目のデータ型や表示名などを定義した<font color="red">**metadata-def.json**</font>を使って、RDEにメタデータを登録することができます。

今回はこれらのJSONファイルの詳細はご説明しませんが、ご参考までに本資料の末尾に掲載します。

## グラフをPNG画像としてRDEに登録する
plot()メソッドでグラフの描画を行えますが、これを画像として保存するときは、Matplotlibのsavefig関数を使用します。


In [ ]:
import hyperspy.api as hs

# 画像保存のためにMatplotlibを利用
import matplotlib.pyplot as plt

s = hs.load("Fei_HAADF-DE_location.dm3")
s.plot()

# プロット結果をPNG画像で保存
# RDEに登録するときはmain_imageディレクトリに配置する必要がある
plt.savefig("graph.png")

作成した画像ファイルをmain_imageというディレクトリに格納することで、RDEに登録してサムネイル表示することができます。

(上記のプログラムはサンプルであるため、カレントディレクトリの直下に保存しています。)

【ご参考】

昨年度のワークショップや、初級者向けセミナーでご紹介しているとおり、Matplotlibには「Axesインタフェース」と「pyplotインタフェース」という2種類のインタフェースがあり、今回は「pyplotインタフェース」を使用しています。

可能であれば「Axesインタフェース」の方が望ましいのですが、確認した限り、HyperSpyと組み合わせる場合は「pyplotインタフェース」を使うしか手段がないようです。

# データ登録のデモンストレーション
それでは、実際にRDEに「Fei_HAADF-DE_location.dm3」をアップロードしてみます。

講師の画面をご覧ください。

# [ご参考] メタデータJSONファイル
前述したmetadata-jsonおよびmetadata-def.jsonを、ご参考までに掲載します。

### metadata.json
https://github.com/tendo-sms/python_intermediate_2023/raw/main/file_4/metadata.json

### metadata-def.json
https://github.com/tendo-sms/python_intermediate_2023/raw/main/file_4/metadata-def.json